In [ ]:
import os
import cv2

def normalize_coordinates(label_path, image_path):
    # Read the image to get its dimensions
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image not found: {image_path}")
        return

    height, width, _ = image.shape

    with open(label_path, 'r') as file:
        lines = file.readlines()

    normalized_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        center_x = float(parts[1])
        center_y = float(parts[2])
        bbox_width = float(parts[3])
        bbox_height = float(parts[4])

        # Normalize coordinates
        center_x /= width
        center_y /= height
        bbox_width /= width
        bbox_height /= height

        # Ensure coordinates are between 0 and 1
        if 0 <= center_x <= 1 and 0 <= center_y <= 1 and 0 <= bbox_width <= 1 and 0 <= bbox_height <= 1:
            normalized_line = f"{class_id} {center_x} {center_y} {bbox_width} {bbox_height}\n"
            normalized_lines.append(normalized_line)
        else:
            print(f"Skipping corrupt label: {label_path}")

    with open(label_path, 'w') as file:
        file.writelines(normalized_lines)

# Path to the base directory containing train, test, and val folders
base_dir = 'C:/Users/aayud/OneDrive/Desktop/VS Code/Web Dev/TSEC Hacks Artemis/mlserver/wildlife_detection/AOD-20250129T035709Z-002/AOD/data'
sets = ['train', 'val']

for dataset in sets:
    image_dir = os.path.join(base_dir, dataset, 'images')
    label_dir = os.path.join(base_dir, dataset, 'labels')

    # Normalize coordinates in all label files
    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        image_path = os.path.join(image_dir, label_file.replace('.txt', '.jpg'))  # Assuming image files are .jpg
        normalize_coordinates(label_path, image_path)

In [ ]:
import os
import cv2
import random
import matplotlib.pyplot as plt

def load_labels(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    labels = []
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        center_x = float(parts[1])
        center_y = float(parts[2])
        bbox_width = float(parts[3])
        bbox_height = float(parts[4])
        labels.append((class_id, center_x, center_y, bbox_width, bbox_height))
    return labels

def draw_bounding_boxes(image, labels):
    height, width, _ = image.shape
    for label in labels:
        class_id, center_x, center_y, bbox_width, bbox_height = label
        x1 = int((center_x - bbox_width / 2) * width)
        y1 = int((center_y - bbox_height / 2) * height)
        x2 = int((center_x + bbox_width / 2) * width)
        y2 = int((center_y + bbox_height / 2) * height)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, str(class_id), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return image

def visualize_samples(base_dir, sets, num_samples=5):
    for dataset in sets:
        image_dir = os.path.join(base_dir, dataset, 'images')
        label_dir = os.path.join(base_dir, dataset, 'labels')

        image_files = os.listdir(image_dir)
        label_files = os.listdir(label_dir)

        # Ensure there are corresponding labels and images
        image_files = [f for f in image_files if f.replace('.jpg', '.txt') in label_files]
        label_files = [f for f in label_files if f.replace('.txt', '.jpg') in image_files]

        if len(image_files) == 0 or len(label_files) == 0:
            print(f"No corresponding images and labels found in {dataset} set.")
            continue

        sample_files = random.sample(image_files, min(num_samples, len(image_files)))

        for image_file in sample_files:
            image_path = os.path.join(image_dir, image_file)
            label_path = os.path.join(label_dir, image_file.replace('.jpg', '.txt'))  # Assuming image files are .jpg

            if not os.path.exists(image_path):
                print(f"Image not found: {image_path}")
                continue

            if not os.path.exists(label_path):
                print(f"Label not found: {label_path}")
                continue

            image = cv2.imread(image_path)
            if image is None:
                print(f"Failed to read image: {image_path}")
                continue

            labels = load_labels(label_path)
            image_with_boxes = draw_bounding_boxes(image, labels)

            plt.figure(figsize=(10, 10))
            plt.imshow(cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB))
            plt.title(f"{dataset} - {image_file}")
            plt.axis('off')
            plt.show()

# Path to the base directory containing train, test, and val folders
base_dir = 'C:/Users/aayud/OneDrive/Desktop/VS Code/Web Dev/TSEC Hacks Artemis/mlserver/wildlife_detection/AOD-20250129T035709Z-002/AOD/data'
sets = ['train', 'val']

# Visualize samples from train and val sets
visualize_samples(base_dir, sets, num_samples=5)

In [6]:
from ultralytics import YOLO

# Initialize the model
model = YOLO('models/yolov8n.pt')

res = model.train(data='./AOD-20250129T035709Z-002/AOD/data/aod.yaml', epochs=1, batch=25, imgsz=640)

In [4]:
import os

def is_label_file_corrupt(label_path):
    try:
        with open(label_path, 'r') as file:
            lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                return True
            class_id = int(parts[0])
            center_x = float(parts[1])
            center_y = float(parts[2])
            bbox_width = float(parts[3])
            bbox_height = float(parts[4])
        return False
    except Exception as e:
        print(f"Error reading label file {label_path}: {e}")
        return True

# Path to the base directory containing train, test, and val folders
base_dir = 'C:/Users/aayud/OneDrive/Desktop/VS Code/Web Dev/TSEC Hacks Artemis/mlserver/wildlife_detection/AOD-20250129T035709Z-002/AOD/data'
sets = ['train', 'val', 'test']

for dataset in sets:
    label_dir = os.path.join(base_dir, dataset, 'labels')

    # Identify and remove corrupt label files
    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        if is_label_file_corrupt(label_path):
            print(f"Removing corrupt label file: {label_path}")
            os.remove(label_path)

## New Dataset

In [2]:
# Importing necessary libraries
import os
import shutil
import cv2
import random
import matplotlib.pyplot as plt
import numpy as np
import copy
from ultralytics import YOLO

In [3]:
# Defining data path and listing its contents
ip_datapath='./data'
os.listdir(ip_datapath)

['buffalo', 'elephant', 'rhino', 'zebra']

In [4]:
image_size=640

In [5]:
# Creating paths for separate images and labels
curr_path=os.getcwd()
imgtrainpath = os.path.join(curr_path,'images','train')
imgvalpath=os.path.join(curr_path,'images','validation')
imgtestpath=os.path.join(curr_path,'images','test')

labeltrainpath=os.path.join(curr_path,'labels','train')
labelvalpath=os.path.join(curr_path,'labels','validation')
labeltestpath=os.path.join(curr_path,'labels','test')

# Creating directories for all paths defined
os.makedirs(imgtrainpath)
os.makedirs(imgvalpath)
os.makedirs(imgtestpath)
os.makedirs(labeltrainpath)
os.makedirs(labelvalpath)
os.makedirs(labeltestpath)

In [6]:
for dirname in os.listdir(ip_datapath):
    dirpath=os.path.join(ip_datapath, dirname)
    for file in os.listdir(dirpath):
        filepath=os.path.join(dirpath, file)
        newname=dirname+'_'+file
        if file.endswith((".txt")): # if label file, take it to label train path
            shutil.copy(filepath, labeltrainpath)
            path=os.path.join(labeltrainpath, file)
            newpath=os.path.join(labeltrainpath, newname)
        elif file.endswith((".jpg", ".JPG")): # if image file, resize and take it to image train path
            img_resized=cv2.resize(cv2.imread(filepath), (image_size, image_size))
            path=os.path.join(imgtrainpath, file)
            cv2.imwrite(path, img_resized)
            newpath=os.path.join(imgtrainpath, newname)
        os.rename(path, newpath) # Rename the file (label or image)

In [7]:
len(os.listdir(imgtrainpath)), len(os.listdir(labeltrainpath))

(1504, 1504)

In [8]:
# function to check if all elements in the list are actually the same
def are_all_elements_same(lst):
    if not lst:
        return True  # An empty list is considered to have all elements the same.

    first_element = lst[0]
    for element in lst[1:]:
        if element != first_element:
            return False

    return True

In [9]:
for file in os.listdir(labeltrainpath):
    classes_list=[]
    with open(os.path.join(labeltrainpath, file), "r") as f:
        for line in f:
            class_id,_,_,_,_=line.strip().split(" ")
            classes_list.append(class_id) # creating list of all unique animal types in given image
            
    # Checking if different types of animals are present in image
    if not are_all_elements_same(classes_list): 
        filepath=os.path.join(labeltrainpath, file)
        newpath=os.path.join(labeltestpath, file)
        shutil.move(filepath, newpath) # moving label file to test path
        basename=os.path.splitext(file)[0]
        print(basename) # printing the image name
        imgfilename=basename+'.jpg'
        oldimgfilepath=os.path.join(imgtrainpath, imgfilename)
        newimgfilepath=os.path.join(imgtestpath, imgfilename) 
        shutil.move(oldimgfilepath, newimgfilepath) # moving image to test path

buffalo_273
buffalo_291
buffalo_297
buffalo_298
buffalo_299
buffalo_300
buffalo_301
elephant_166
elephant_288
elephant_290
elephant_291
elephant_345
rhino_238
rhino_256
rhino_375
zebra_073
zebra_338
zebra_339


In [10]:
plt.figure(figsize=(30,30))
for i in range(6):
    test_image=os.path.join(imgtestpath, os.listdir(imgtestpath)[i])
    ax=plt.subplot(3,2,i+1)
    
    # Display actual image
    plt.imshow(cv2.imread(test_image)) 
    plt.xticks([])
    plt.yticks([])


In [11]:
# Checking the size of test dataset
len(os.listdir(imgtestpath)), len(os.listdir(labeltestpath))

# Checking the size of training(+validation) dataset
len(os.listdir(imgtrainpath)), len(os.listdir(labeltrainpath))

(1486, 1486)

In [12]:
# moving 20% of data to validation

factor=0.2 

for file in random.sample(os.listdir(imgtrainpath), int(len(os.listdir(imgtrainpath))*factor)):
    basename=os.path.splitext(file)[0]
    textfilename=basename+'.txt'
    labelfilepath=os.path.join(labeltrainpath, textfilename)
    labeldestpath=os.path.join(labelvalpath, textfilename)
    imgfilepath=os.path.join(imgtrainpath, file)
    imgdestpath=os.path.join(imgvalpath, file)
    shutil.move(imgfilepath, imgdestpath)
    shutil.move(labelfilepath, labeldestpath)

In [13]:
len(os.listdir(imgtrainpath)), len(os.listdir(labeltrainpath))
len(os.listdir(imgvalpath)), len(os.listdir(labelvalpath))

(297, 297)

In [14]:
# function to obtain bounding box coordinates from text label files
def get_bbox_from_label(text_file_path):
    bbox_list=[]
    with open(text_file_path, "r") as file:
        for line in file:
            class_id,x_centre,y_centre,width,height=line.strip().split(" ")
            x1=(float(x_centre)+(float(width)/2))*image_size
            x0=(float(x_centre)-(float(width)/2))*image_size
            y1=(float(y_centre)+(float(height)/2))*image_size
            y0=(float(y_centre)-(float(height)/2))*image_size
            
            vertices=np.array([[int(x0), int(y0)], [int(x1), int(y0)], 
                               [int(x1),int(y1)], [int(x0),int(y1)]])
            bbox_list.append(vertices)      
    return tuple(bbox_list)

In [15]:
# defining red color in RGB to draw bounding box
red=(255,0,0) 
# defining newline variable for config file
newline='\n'

In [16]:
# Drawing bounding box for random images in training data
plt.figure(figsize=(30,30))
for i in range(1,8,2):
    k=random.randint(0, len(os.listdir(imgtrainpath))-1)
    img_path=os.path.join(imgtrainpath, sorted(os.listdir(imgtrainpath))[k])
    label_path=os.path.join(labeltrainpath, sorted(os.listdir(labeltrainpath))[k])
    bbox=get_bbox_from_label(label_path) # extracting bounding box coordinates
    image=cv2.imread(img_path)
    image_copy=copy.deepcopy(image)
    ax=plt.subplot(4, 2, i)
    plt.imshow(image) # displaying image
    plt.xticks([])
    plt.yticks([])
    cv2.drawContours(image_copy, bbox, -1, red, 2) # drawing bounding box on copy of image
    ax=plt.subplot(4, 2, i+1)
    plt.imshow(image_copy) # displaying image with bounding box
    plt.xticks([])
    plt.yticks([])

In [17]:
# Starting with a comment in config file
ln_1='# Train/val/test sets'+newline

# train, val and test path declaration
ln_2='train: ' +"'"+imgtrainpath+"'"+newline
ln_3='val: ' +"'" + imgvalpath+"'"+newline
ln_4='test: ' +"'" + imgtestpath+"'"+newline
ln_5=newline
ln_6='# Classes'+newline

# names of the classes declaration
ln_7='names:'+newline
ln_8='  0: buffalo'+newline
ln_9='  1: elephant'+newline
ln_10='  2: rhino'+newline
ln_11='  3: zebra'

config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8, ln_9, ln_10, ln_11]

In [18]:
# Creating path for config file
config_path=os.path.join(curr_path, 'config.yaml')
config_path

'c:\\Users\\aayud\\OneDrive\\Desktop\\VS Code\\Web Dev\\TSEC Hacks Artemis\\mlserver\\wildlife_detection\\config.yaml'

In [19]:
# Writing config file
with open(config_path, 'w') as f:
    f.writelines(config_lines)

In [20]:
# Using YOLO's pretrained model architecture and weights for training
model=YOLO('yolov8m.yaml')


                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.Conv                  [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.Conv                  [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.C2f                   [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.Conv                  [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.C2f                   [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.Conv                  [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.C2f                   [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.Conv                  [384

In [21]:
#  Training the model
import torch
results=model.train(data=config_path, epochs=20)
model.save('wildlife_detector.pt')

Ultralytics YOLOv8.0.43  Python-3.11.5 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
yolo\engine\trainer: task=detect, mode=train, model=C:\Users\aayud\AppData\Roaming\Python\Python311\site-packages\ultralytics\models\v8\yolov8m.yaml, data=c:\Users\aayud\OneDrive\Desktop\VS Code\Web Dev\TSEC Hacks Artemis\mlserver\wildlife_detection\config.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, min_memory=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_st

In [ ]:
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load the trained model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = YOLO('wildlife_detector.pt')  # Replace 'best_model.pt' with the path to your trained model

metrics = model.val(data=config_path, device=device)

# Print the evaluation metrics
print(metrics)

# Visualize predictions on a few validation images
def visualize_predictions(model, image_paths):
    for image_path in image_paths:
        results = model(image_path)
        results.show()

# List of validation images to visualize
val_image_paths = [
    'images/test/elephant_290.jpg',
    'images/test/buffalo_291.jpg',
    'images/test/rhino_256.jpg',
]